In [1]:
#adding communication through seial

In [ ]:
import logging
import time
import mido
import numpy as np
from numpy import interp
from IPython.display import clear_output
from threading import Thread
import random
import tkinter as tk
import simpleaudio as sa

In [2]:
class MIDIDispatcher(Thread):
    def __init__(self, inport):
        super().__init__()
        self.inport = inport
        self.latest_msgs = {}
        self.cc_map = {}
        self.running = False

    def register(self, receiver_id, receiver_cc_map):
        '''
        Register a midi receiver:
        - receiver_id: like 0, 'id0' or whatever
            note: special id '*' means master (send to all receivers)
        - receiver_cc_map: dict {'message_key': cc_num, ...} like {'vol': 7, ...}
        '''
        self.latest_msgs[receiver_id] = {}
        for key, cc in receiver_cc_map.items():
            # for now we assume that there is only one receiver for every CC
            if cc in self.cc_map:
                logging.warning(f"Overwriting mapping for CC {cc}")
                logging.warning(f"Old recv_id: {self.cc_map[cc][0]}, key: {self.cc_map[cc][1]}")
                logging.warning(f"New recv_id: {receiver_id}, key: {key}")
            self.cc_map[cc] = [receiver_id, key]

    def on_msg(self, msg):
        cc = msg.control
        if cc not in self.cc_map:
            return

        recv_id, msg_key = self.cc_map[cc]
        if recv_id == '*':
            all_recv_ids = self.latest_msgs.keys()
            for recv_id in all_recv_ids:
                self.latest_msgs[recv_id][msg_key] = msg
        else:
            self.latest_msgs[recv_id][msg_key] = msg

    def get_messages(self, receiver_id, flush=True):
        msgs = self.latest_msgs[receiver_id].copy()
        if flush:
            self.latest_msgs[receiver_id] = {}
        return msgs

    def run(self):
        self.running = True
        while self.running:
            msg = self.inport.receive()
            self.on_msg(msg)

    def stop(self):
        self.running = False


In [3]:
class Metronome():
    global master_controls
    global metronomes_controls

    def __init__(self, midi_dispatcher, inst_number, vol_beat=1, tempo=60, active=True, min_tempo = 20, max_tempo = 200, beat_freq = 1000, beat_dur = 0.1):
        self.midi_dispatcher = midi_dispatcher
        self.inst_number = inst_number
        self.tempo = tempo
        self.active = active
        self.on_off_list = ['ON','OFF','OFF','ON']
        self.selector = 0
        self.tapped_list = []
        self.tapped_tempo = 0
        self.sync_on_off = ['OFF','ON','ON','OFF']
        self.sync_on_off_selector = 0
        #selecting specific MIDI controls for this metronome
        self.controls = {key: val[self.inst_number] for key, val in metronomes_controls.items()}
        self.midi_dispatcher.register(self.inst_number, self.controls)
        self.min_tempo = min_tempo
        self.max_tempo = max_tempo
        self.vol_beat = vol_beat
        self.min_vol = 0.01
        self.max_vol = 1
        self.beat_freq = beat_freq #possible dev. change pitch for each metronome.
        self.beat_dur = beat_dur #possible development of the code, change duration of the beat
        self.fs = 44100 #sampling size
        self.t = np.linspace(0, self.beat_dur, int(self.beat_dur*self.fs), False) 
        self.note = np.sin(self.beat_freq *self.t *2 *np.pi)
        self.audio = self.note*(2**15-1) / np.max(np.max(self.note)) #16 bit conversion
        self.audio = self.audio.astype(np.int16)
        self.beat_sound = sa.WaveObject(self.audio,1,2,self.fs)
        self.to_sync = False
        
    def __str__(self):
        return f"Instrument number: {self.inst_number}\nTempo: {self.tempo}\n"
   
    def beat(self):
        if self.active:
            self.beat_sound.play()
            time.sleep(60/self.tempo)
        self.update()
            
    def set_volume(self, b_sound, v_beat):
        if v_beat < 0:
            v_beat = self.min_vol
        if v_beat > 1:
            v_beat = self.max_vol
        audio_data = np.frombuffer(b_sound.audio_data, dtype=np.int16)
        volume_factor = v_beat * 32767 / np.max(np.abs(audio_data))
        audio_data = np.int16(audio_data * volume_factor)
        self.beat_sound = sa.WaveObject(audio_data, b_sound.num_channels, b_sound.bytes_per_sample, b_sound.sample_rate)
    
    def tap_tempo(self):
        tapped_tempo_list = self.tapped_list[:-6:-1] #just save at least the last 5 inputs 
        tapped_tempo_list.reverse()
        self.tapped_tempo = (60*len(tapped_tempo_list)) / (tapped_tempo_list[-1] - tapped_tempo_list[0])
        
    def tap_me(self):
        self.tempo = int(self.tapped_tempo)
        print(f'The estimated tempo is {self.tapped_tempo} BPM.')
        self.tapped_list = [] #cleaning the original
    
    def select_to_sync(self):
        sync_list.add(self.inst_number)
        sync_tempo_list.add(self.tempo)
        self.to_sync = True
        print(f"selected to sync: {sync_list}")
        print(f"List of tempos to sync: {sync_tempo_list}")
        
    def unselect_to_sync(self):
        sync_list.remove(self.inst_number)
        self.to_sync = False
        try:
            sync_tempo_list.remove(self.tempo)
        except:
            print('Tempo to unselect not on list')
        print(f"selected to sync: {sync_list}")
        print(f"List of tempos to sync: {sync_tempo_list}")
              
    def sync_max(self):
        if len(sync_tempo_list) != 0:
            return max(sync_tempo_list)
    def sync_min(self):
        if len(sync_tempo_list) != 0:
            return min(sync_tempo_list)
    def sync_avg(self):
        if len(sync_tempo_list) != 0:
            return np.mean(list(sync_tempo_list))
    def sync_rand(self):
        if len(sync_tempo_list) != 0:
            selector = random.randrange(len(sync_tempo_list))
            return list(sync_tempo_list)[selector]
        
    def sync_me(self, new_tempo):
        self.tempo = new_tempo
        sync_list.remove(self.inst_number)
        if len(sync_list) == 0:
            self.clear_sync_q()
                
    def clear_sync_q(self):
        sync_list.clear()
        sync_tempo_list.clear()
        self.sync_on_off_selector = 0 #fixing bug to unselect after hard cleaning func
        print(f"Sync Queu Clear: {sync_list} | {sync_tempo_list}")
    
    def tempo_change(self, t_change):
        if t_change == True:
            self.tempo = self.tempo + 1
            print(f"Inst: {self.inst_number} Tempo: {self.tempo} \n", end='\r')
        elif t_change == False:
            self.tempo = self.tempo - 1
            print(f"Inst: {self.inst_number} Tempo: {self.tempo} \n", end='\r')

    def update(self):
        global sync_list
        global sync_tempo_list
        global sync_selector
        global sync_mode_selector
        global sync_mode_selected
        global master_tempo
        global master_onoff
        global master_vol
        global master_tap
        global master_select
        
        latest_messages = self.midi_dispatcher.get_messages(self.inst_number)
        
        for key, msg in latest_messages.items():
            if key == 'tempo_knob':
                self.tempo = int(interp(msg.value,[0,127],[self.min_tempo,self.max_tempo]))
                #clear_output()
                print(f"Inst: {self.inst_number} Tempo: {self.tempo}", end='\r')
                if msg.control == 23:
                    master_tempo = int(interp(msg.value,[0,127],[self.min_tempo,self.max_tempo]))
  
            elif key == 'tap_button':
                self.tapped_list.append(time.time())
                try:
                    self.tap_tempo()
                except:
                    continue
                if msg.control == 71:
                    master_tap = self.tapped_tempo                    
            
            elif key == 'tap_metronome':
                if self.tapped_list != []:
                    self.tap_me()
                else:
                    print('Empty tap tempo list')
                print(f"Metronome number: {self.inst_number} | Tempo: {self.tempo}\n", end='\r')
            
            elif key == 'vol_slide':
                if msg.value == 0:
                    self.active = False
                if msg.value != 0:
                    self.active = True
                    self.vol_beat = interp(msg.value,[0,127],[self.min_vol,self.max_vol])
                    self.set_volume(self.beat_sound, self.vol_beat)
                    clear_output()
                    print(f"Inst: {self.inst_number} vol: {self.vol_beat*100}")
                    if msg.control == 7:
                        master_vol = self.vol_beat
                    
            elif key == 'play_stop':
                self.selector += 1
                if self.selector >3:
                    self.selector = 0
                if 'ON' in self.on_off_list[self.selector]:
                    clear_output()
                    print(f"Inst: {self.inst_number} 'ON'")
                    self.active = True
                if 'OFF' in self.on_off_list[self.selector]:
                    clear_output()
                    print(f"Inst: {self.inst_number} 'OFF'")
                    self.active = False
            
            elif key == 'down_beat':
                self.active = False
                self.active = True
                print("Down Beat!")
                
            elif key == 'sync_select':
                self.sync_on_off_selector += 1
                if self.sync_on_off_selector >3:
                    self.sync_on_off_selector = 0
                try:
                    if 'ON' in self.sync_on_off[self.sync_on_off_selector]:
                        self.select_to_sync()
                    elif 'OFF' in self.sync_on_off[self.sync_on_off_selector]:
                        self.unselect_to_sync()
                except:
                    continue
                if msg.control == 55:
                    if master_select == False:
                        master_select = True
                    else:
                        master_select = False
                    
            elif key == 'sync_mode_plus':
                sync_selector += 1
                if sync_selector > 6:
                    sync_selector = 6
                sync_mode_selected = sync_mode_selector[sync_selector]
                print(f"Sync Mode: {sync_mode_selected}", end='r')
            elif key == 'sync_mode_minus':
                sync_selector -= 1
                if sync_selector < 0:
                    sync_selector = 0
                sync_mode_selected = sync_mode_selector[sync_selector]
                print(f"Sync Mode: {sync_mode_selected}", end='r')
                
            elif key == 'sync_selected':
                if self.inst_number in sync_list:
                    if sync_mode_selected == 'MAX':
                        tempo = self.sync_max()
                        self.sync_me(tempo)
                    elif sync_mode_selected == 'MIN':
                        tempo = self.sync_min()
                        self.sync_me(tempo)
                    elif sync_mode_selected == 'AVG':
                        tempo = self.sync_avg()
                        self.sync_me(tempo)
                    elif sync_mode_selected == 'RAND':
                        tempo = self.sync_rand()
                        self.sync_me(tempo)
                print(f"Metronome number: {self.inst_number} | Tempo: {self.tempo}\n") #, end='\r')
                
            elif key == 'clear_sync_q':
                self.clear_sync_q()
            
            elif key == 'tempo_minus':
                 if self.inst_number in sync_list:
                    self.tempo_change(False)
            elif key == 'tempo_plus':
                 if self.inst_number in sync_list:
                    self.tempo_change(True)

In [4]:
def create_metronomes(metro_func, midi_dispatcher):
    start = True
    while start:
            try:
                num_inst = int(input("How many metronomes (1-7): "))
                if num_inst >= 1 and num_inst <= 8:
                    start = False
                else:
                    print("Wrong Input. Try again")
                    continue
            except:
                print("Wrong Input. Try again")      
    inst_list = list(range(num_inst))
    metronomes = [metro_func(midi_dispatcher, i) for i in inst_list] 
    return metronomes

def create_thread(metronome):
    def inner():
        while True:
            metronome.beat()
    return Thread(target= inner)


In [5]:
#---UI---

In [6]:
font=("Press Start 2P", 10)

In [7]:
class GlobalLabels():
    def __init__(self, frame):
        self.frame = tk.Frame(frame, bg="black", width=250)

        self.sync_list_label = tk.Label(self.frame, text=f"sync_list:\n{sync_list}", fg="white", bg="black", font=font)
        self.sync_tempo_list_label = tk.Label(self.frame, text=f"sync_tempo_list:\n{sync_tempo_list}", fg="white", bg="black", font=font)
        self.sync_mode_selected_label = tk.Label(self.frame, text=f"sync_mode_selected: {sync_mode_selected}", fg="white", bg="black", font=font)

        self.sync_list_label.pack(side=tk.TOP, pady=5)
        self.sync_tempo_list_label.pack(side=tk.TOP, pady=5)
        self.sync_mode_selected_label.pack(side=tk.TOP, pady=5)

        self.frame.grid(row=0, column=0)

In [8]:
class MetronomeUI():
    def __init__(self, master, metronome, global_labels):
        self.metronome = metronome
        self.frame = tk.Frame(master, bg="black")
        self.frame.config(width=150)

        self.global_labels = global_labels

        self.inst_label = tk.Label(self.frame, text=f"m. {self.metronome.inst_number}", fg="white", bg="black", font=("pixelmix", 10))
        self.tempo_label = tk.Label(self.frame, text=f"t. {self.metronome.tempo}", fg="white", bg="black", font=("pixelmix", 10))
        self.vol_label = tk.Label(self.frame, text=f"v. {self.metronome.vol_beat*100:.0f}%", fg="white", bg="black", font=("pixelmix", 10))
        self.status_label = tk.Label(self.frame, text="Status: ON", fg="white", bg="black", font=("pixelmix", 10), width=10)
        self.to_sync = tk.Label(self.frame, text="Sync ON" if self.metronome.to_sync else "Sync OFF", fg="white", bg="black", font=("pixelmix", 10))
        self.tap_label = tk.Label(self.frame, text=f"tap. {int(self.metronome.tapped_tempo)}", fg="white", bg="black", font=("pixelmix", 10))

        self.inst_label.pack(side=tk.TOP, pady=5)
        self.tempo_label.pack(side=tk.TOP, pady=5)
        self.vol_label.pack(side=tk.TOP, pady=5)
        self.status_label.pack(side=tk.TOP, pady=5)
        self.to_sync.pack(side=tk.TOP, pady=5)
        self.tap_label.pack(side=tk.TOP, pady=5)

        self.frame.grid(row=0, column=self.metronome.inst_number+1, padx=10)

    def update(self):
        # Update labels with global variables information
        self.global_labels.sync_list_label.config(text=f"sync_list:\n{sync_list}", fg="white", bg="black", font=font)
        self.global_labels.sync_tempo_list_label.config(text=f"sync_tempo_list:\n{sync_tempo_list}", fg="white", bg="black", font=font)
        self.global_labels.sync_mode_selected_label.config(text=f"sync_mode_selected: {sync_mode_selected}", fg="white", bg="black", font=font)
        # Update labels with current metronome information
        self.tempo_label.config(text=f"t. {self.metronome.tempo}", fg="white", bg="black", font=font)
        self.vol_label.config(text=f"v. {self.metronome.vol_beat*100:.0f}%", fg="white", bg="black", font=font)
        self.status_label.config(text="Status: ON" if self.metronome.active else "Status: OFF", fg="white", bg="black", font=font)
        self.to_sync.config(text="Sync ON" if self.metronome.to_sync else "Sync OFF", fg="white", bg="black", font=font)
        self.tap_label.config(text=f"tap. {int(self.metronome.tapped_tempo)}", fg="white", bg="black", font=font)

In [9]:
def tkinter_ui(metronomes):
    root = tk.Tk()
    root.title("Metronomes")
    
    # Create a GlobalLabels object
    global_labels = GlobalLabels(root)

    # Create MetronomeUI objects for each metronome
    metronome_uis = [MetronomeUI(root, metronome, global_labels) for metronome in metronomes]

    def update_ui():
        # Update MetronomeUI objects with current metronome information
        for metronome_ui in metronome_uis:
            metronome_ui.update()
        # Schedule next update in 100 milliseconds
        root.after(50, update_ui)

    # Schedule initial update
    root.after(50, update_ui)

    root.mainloop()

In [10]:
#---run---

In [11]:
 inport = mido.open_input() #check if it has been assigned before

In [12]:
#global variables to sync metronomes to specific tempo functions
sync_list = set()
sync_tempo_list = set()
sync_selector = 0
sync_mode_selector = ['MAX','MIN','MIN','AVG','AVG','RAND','RAND'] #['MAX','MIN','AVG','RAND'] 
sync_mode_selected = 'MAX' #by default
master_tempo = '_' 
master_onoff = False
master_select = False
master_vol = 100
master_tap = 0

In [13]:
#Creating a dictionary to set MIDI controllers for each metronome on their initialization depending on the metronome inst_number 
metronomes_controls = {
          'inst_number':[n + 0 for n in range(0,7)], 
          'vol_slide':[n + 0 for n in range(0,7)],
          'tempo_knob': [n + 16 for n in range(0,7)],
          'play_stop': [n + 32 for n in range(0,7)],
          'sync_select': [n + 48 for n in range(0,7)],
          'tap_button': [n + 64 for n in range(0,7)]} 
#These controls are global. They do not change depending on the metronome inst_number.
master_controls = {'vol_slide': 7, 'tempo_knob': 23, 'sync_select': 55, 'tap_button': 71, 'down_beat':39, 'tempo_minus': 58, 'tempo_plus': 59, # 'play_stop': 39,41
                   'tap_metronome': 45, 'sync_selected': 60, 'sync_mode_minus': 61,'sync_mode_plus':62, 'clear_sync_q':46} 

In [14]:
#start Midi dispatcher
d = MIDIDispatcher(inport)
d.register('*', master_controls)
d.start()

#How many metronomes do you need? only one midi controller at the moment
metronomes = create_metronomes(Metronome, d)

How many metronomes (1-7): 6


In [15]:
# Create the threads
threads = [create_thread(metronome) for metronome in metronomes]
#thread_ui = Thread(target=tkinter_ui, args=[metronomes])

# Start metronomes at (almost) the same time
for t in threads:
    t.start()

#start the UI
tkinter_ui(metronomes)